<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-10 14:28:07
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 2.12 K (0.02%)
Current PnL: -12.97 L (-9.04%)
CY Booked + Current PnL: -6.01 L (-4.19%)
-------------------
Total profit:  4.86 L
Total loss:  -17.83 L
-------------------
Total Booked + Current PnL: 20.97 L (17.99%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.46 L (52.11%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87574.0,-7742.0,7777.0,2.52,-8.12,8.88,0.04,157.0,-1.00,0.64,72.73,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-18.62,H-LC,89.58,110842.0,7276.0,13434.0,-1.06,7.03,12.12,20.00,32.0,0.54,0.81,22.09,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,73.96,108192.0,3880.0,13654.0,0.33,3.72,12.62,16.81,27.0,0.28,0.79,20.36,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,7.29,128682.0,-13797.0,14412.0,0.25,-9.68,11.20,0.44,247.0,-0.96,0.94,51.88,OX40N,NTT,BANKS
71,SBIN,760.30,863.00,-11.51,M-LC,42.71,214266.0,12786.0,14420.0,-0.30,6.35,6.73,13.51,77.0,0.89,1.56,19.74,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,41.67,151666.0,753.0,59696.0,0.00,0.50,39.36,40.06,103.0,0.01,1.11,24.05,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,34.38,99725.0,-779.0,31423.0,1.04,-0.77,31.51,30.49,97.0,-0.02,0.73,31.38,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.00,-21.26,M-SC,51.04,164100.0,-3675.0,59519.0,6.54,-2.19,36.27,33.28,206.0,-0.06,1.20,27.05,XY24,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.00,-169.32,M-SC,84.38,79500.0,-22710.0,116102.0,3.60,-22.22,146.04,91.37,212.0,-0.20,0.58,29.05,XY25,NTT,FINANCE
65,RELAXO,902.64,1176.00,-26.70,H-SC,97.40,66240.0,-49298.0,84290.0,3.60,-42.67,127.25,30.28,131.0,-0.58,0.48,29.41,X40N,NTT,FOOTWEAR
8,AWL,325.39,485.00,-67.84,H-MC,26.04,223451.0,-44996.0,176683.0,3.08,-16.76,79.07,49.05,98.0,-0.25,1.63,14.89,XY24,NTT,FMCG
54,KPIGREEN,497.21,731.79,30.70,H-SC,72.92,133925.0,8628.0,50490.0,2.73,6.89,37.70,47.18,143.0,0.17,0.98,67.55,MH,ATH,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,208.79,M-SC,77.08,137933.0,-45145.0,124126.0,-2.45,-24.66,89.99,43.14,217.0,-0.36,1.01,29.75,XY24,NTT,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,15.62,48480.0,7225.0,3583.0,-1.66,17.51,7.39,26.19,272.0,2.02,0.35,14.14,XY24,ATH,IT
50,JCHAC,2282.24,2282.00,19726.09,M-SC,71.88,96835.0,-30970.0,30958.0,-1.54,-24.23,31.97,-0.01,222.0,-1.00,0.71,9.65,OX40N,NTT,AC
77,STARHEALTH,551.56,761.00,-5.68,M-SC,6.25,154456.0,-45760.0,121789.0,-1.54,-22.86,78.85,37.97,215.0,-0.38,1.13,24.02,XY24,NTT,INSURANCE
12,BANKINDIA,116.91,190.00,-33.74,M-MC,10.42,175578.0,-4230.0,116636.0,-1.51,-2.35,66.43,62.52,173.0,-0.04,1.28,30.93,XR,NTT,BANKS


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.42,H-SC,98.96,87574.0,-7742.0,7777.0,2.52,-8.12,8.88,0.04,157.0,-1.00,0.64,72.73,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,7.29,128682.0,-13797.0,14412.0,0.25,-9.68,11.20,0.44,247.0,-0.96,0.94,51.88,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.80,M-SC,32.29,151894.0,-19247.0,19184.0,0.53,-11.25,12.63,-0.03,191.0,-1.00,1.11,9.17,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,90.62,148775.0,-25784.0,25812.0,-0.04,-14.77,17.35,0.02,133.0,-1.00,1.09,29.01,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.49,X-MC,5.21,85491.0,-17161.0,17158.0,-0.74,-16.72,20.07,-0.00,59.0,-1.00,0.62,29.96,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,30.21,263952.0,-31982.0,125905.0,0.01,-10.81,47.70,31.73,1.0,-0.25,1.93,5.65,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.91,X-LC,59.38,289085.0,27484.0,118149.0,-1.14,10.51,40.87,55.67,2.0,0.23,2.11,17.25,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.79,X-LC,64.58,270099.0,-248.0,34870.0,-0.49,-0.09,12.91,12.81,5.0,-0.01,1.97,12.96,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.95,X-LC,78.12,270618.0,41385.0,25898.0,0.02,18.05,9.57,29.35,10.0,1.60,1.97,28.58,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-6.10,X-LC,11.46,79430.0,-30742.0,119820.0,-1.28,-27.90,150.85,80.85,11.0,-0.26,0.58,16.99,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3129.4,NaN,NaN,95.83,78235.0,43786.0,0.0,0.09,127.11,0.00,127.11,274.0,inf,0.57,19.63,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-13.94,L-SC,100.00,172589.0,30747.0,25819.0,1.88,21.68,14.96,39.88,262.0,1.19,1.26,74.90,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.74,X-SC,91.67,250919.0,45442.0,35881.0,1.67,22.12,14.30,39.58,65.0,1.27,1.83,34.46,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,60.42,231953.0,6991.0,160674.0,0.61,3.11,69.27,74.53,189.0,0.04,1.69,34.67,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,30.70,H-SC,72.92,133925.0,8628.0,50490.0,2.73,6.89,37.70,47.18,143.0,0.17,0.98,67.55,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,38.54,117432.0,15428.0,29593.0,0.08,15.12,25.20,44.14,126.0,0.52,0.86,33.82,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.05,H-SC,93.75,179910.0,15240.0,53847.0,-0.58,9.25,29.93,41.95,124.0,0.28,1.31,17.41,X40N,ATH,AC
95,WIPRO,243.46,420.00,-4.22,M-LC,79.17,164331.0,13386.0,96068.0,-1.03,8.87,58.46,72.51,71.0,0.14,1.20,13.20,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3129.40,NaN,NaN,95.83,78235.0,43786.0,0.0,0.09,127.11,0.00,127.11,274.0,inf,0.57,19.63,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.58,M-SC,58.33,195951.0,24330.0,137734.0,-1.11,14.18,70.29,94.43,201.0,0.18,1.43,44.62,XR,NTT,DURABLES
54,KPIGREEN,497.21,731.79,30.70,H-SC,72.92,133925.0,8628.0,50490.0,2.73,6.89,37.70,47.18,143.0,0.17,0.98,67.55,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.22,M-LC,79.17,164331.0,13386.0,96068.0,-1.03,8.87,58.46,72.51,71.0,0.14,1.20,13.20,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.33,H-SC,80.21,137249.0,13913.0,119846.0,0.07,11.28,87.32,108.45,123.0,0.12,1.00,36.15,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,66.67,18913.0,-30680.0,0.0,0.38,-61.86,0.00,-61.86,270.0,-inf,0.14,26.83,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,19.79,66816.0,-46733.0,86787.0,1.31,-41.16,129.89,35.27,265.0,-0.54,0.49,63.53,XR,NTT,HOTELS
32,ENRIN,1377.95,3129.40,NaN,NaN,95.83,78235.0,43786.0,0.0,0.09,127.11,0.00,127.11,274.0,inf,0.57,19.63,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7222.22,M-SC,65.62,80362.0,-13448.0,90062.0,2.30,-14.34,112.07,81.67,245.0,-0.15,0.59,55.63,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87574.0,-7742.0,7777.0,2.52,-8.12,8.88,0.04,157.0,-1.00,0.64,72.73,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.94,L-SC,100.00,172589.0,30747.0,25819.0,1.88,21.68,14.96,39.88,262.0,1.19,1.26,74.90,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87574.0,-7742.0,7777.0,2.52,-8.12,8.88,0.04,157.0,-1.00,0.64,72.73,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.09,X-SC,97.40,99678.0,-1285.0,95810.0,-0.29,-1.27,96.12,93.63,67.0,-0.01,0.73,48.47,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-26.70,H-SC,97.40,66240.0,-49298.0,84290.0,3.60,-42.67,127.25,30.28,131.0,-0.58,0.48,29.41,X40N,NTT,FOOTWEAR
32,ENRIN,1377.95,3129.40,NaN,NaN,95.83,78235.0,43786.0,0.0,0.09,127.11,0.00,127.11,274.0,inf,0.57,19.63,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.97
1,25,41.96
2,50,70.65


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.74
LC    33.69
MC    21.56
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.13
X40      12.87
XY25     12.06
X40N     11.92
XR        9.96
OX40N     8.21
AR        8.17
MH        1.71
X5K       1.41
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.58
M-SC    15.93
H-LC    14.01
X-LC    12.84
H-MC     9.60
X-MC     7.57
M-LC     5.80
X-SC     4.57
M-MC     4.04
L-SC     2.66
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.25,-0.16,33.65
IT,12.00,-10.51,64.83
FINANCE,8.56,-11.20,51.56
BANKS,7.62,-9.13,53.66
MISC,5.88,-19.59,75.28
PAINTS,5.66,-12.59,34.78
ELECTRICAL,5.45,1.42,45.14
HEALTHCARE,4.75,0.86,28.11
AUTO,4.31,-14.43,64.75


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2625587.0,26
XR,1094190.0,13
AR,1028576.0,10
X40,690655.0,10
X40N,530108.0,11
XY25,466341.0,8
OX40N,306612.0,11
SR,178221.0,2
MH,81913.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1964826.0,20
M-SC,1762401.0,21
H-LC,621482.0,12
X-LC,550445.0,8
H-MC,517436.0,8
M-MC,416133.0,3
X-MC,415539.0,6
X-SC,292207.0,5
M-LC,266267.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      889192.0      6
M-SC       XY24      801582.0      7
H-SC       AR        522419.0      3
M-SC       XR        340254.0      4
M-MC       XY24      299497.0      2
H-MC       XY24      293177.0      3
X-LC       X40       278995.0      3
H-SC       XR        215818.0      3
           X40N      201490.0      4
H-LC       X40N      199417.0      4
M-SC       AR        187811.0      2
           OX40N     165055.0      5
H-LC       X40       159827.0      3
X-MC       X40       156822.0      2
M-LC       XY24      155779.0      2
L-SC       XR        149904.0      2
X-SC       XR        121534.0      1
X-LC       AR        119820.0      1
M-MC       XR        116636.0      1
M-SC       XY25      116102.0      1
X-MC       XY25      105296.0      1
X-LC       XY25      101120.0      3
M-LC       XR         96068.0      1
X-SC       SR         95810.0      1
H-SC       OX40N      85417.0      3
X-MC       XY24       82888.0      1
M-SC       SR         82411.0      1
H-MC       XY25       81999.0      1
H-LC       AR         79917.0      1
           X5K        75205.0      2
M-SC       X40        69186.0      1
H-LC       X200       68927.0      1
H-MC       AR         59696.0      1
L-SC       AR         58913.0      1
L-MC       XR         53976.0      1
X-MC       X40N       53375.0      1
X-LC       X40N       50510.0      1
H-SC       MH         50490.0      1
L-LC       XY25       47404.0      1
X-SC       OX40N      38982.0      2
H-LC       XY24       38189.0      1
X-SC       XY24       35881.0      1
H-MC       MH         31423.0      1
           X40        25825.0      1
L-SC       XY24       25819.0      1
H-MC       X40N       25316.0      1
X-MC       OX40N      17158.0      1
M-LC       XY25       14420.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
